<img width="8%" alt="Google Sheets.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Google%20Sheets.png" style="border-radius: 15%">

# Google Sheets - Update deals database

**Tags:** #googlesheets #gsheet #data #naas_drivers #sales-engine #automation #deals

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook updates "Deals" database.

## Input

### Import libraries

In [ ]:
from naas_drivers import gsheet
import pandas as pd
import os
from datetime import date
import naas_data_product
import time

### Setup variables
**Inputs**
- `entity_dir`: This variable represents the entity directory.
- `entity_name`: This variable holds the entity name.
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name_input`: Google Sheets sheet name storing growth data.

**Outputs**
- `output_dir`: Output directory to save file to.
- `file_sales`: Output file name to save as picke.
- `sheet_name_output`: Google Sheets sheet name storing sales qualified leads.

In [ ]:
# Inputs
entity_dir = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "entity_dir")
entity_name = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "entity_name")
spreadsheet_url = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "abi_spreadsheet")
sheet_name_input = "CONTACTS"

# Outputs
output_dir = os.path.join(entity_dir, "sales-engine", date.today().isoformat())
file_deals = "deals"
sheet_name_output = "DEALS"

## Model

### Get deals

In [ ]:
df_init = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name_output)
if not isinstance(df_init, pd.DataFrame):
    df_init = pd.DataFrame()
print("- Deals (init):", len(df_init))
df_init.head(1)

### Get contacts view

In [ ]:
df_contacts = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name_input)
if not isinstance(df_contacts, pd.DataFrame):
    df_contacts = pd.DataFrame()
print("- Contacts view:", len(df_contacts))
# df_contacts.head(1)

### Get entity data

In [ ]:
df_entity = gsheet.connect(spreadsheet_url).get(sheet_name="ENTITY").fillna("NA")
df_entity = df_entity[df_entity["ENTITY"] == entity_name].reset_index(drop=True)
print("- Entity:", entity_name)
print("- Entity row:", len(df_entity))
# df_entity.head(1)

### Create deals database

In [ ]:
def get_list(df, column):
    filters = []
    values = df.loc[0, column].split(",")
    if len(values) > 0:
        filters = [v.strip() for v in values]
    return filters

def create_db_deals(
    df_init,
    df_contacts,
    df_entity,
    entity_name
):
    # Init
    df = df_contacts.copy()
    crm_ids = get_dict_from_df(df_init, "CRM_DEAL_ID", "DEAL_NAME", "deals_crm_ids", output_dir, force_update=False)
    print("CRM ID's:", len(crm_ids))
    
    # Filters on SQL
    df = df[
        (df["ENTITY"].str.contains(entity_name)) & 
        (df["LEAD_STATUS"].isin(["Sales Qualified Lead"]))
    ]
    print("Sales Qualified Lead:", len(df))
    print()
    
    # Apply ICP filters from entity
    for c in df_entity.columns:
        value = df_entity.loc[0, c]
        if c.startswith("ICP"):
            col_icp = c.replace("ICP_", "")
            if value == "NA":
                filter_icp = df[col_icp].unique().tolist()
            else:
                filter_icp = get_list(df_entity, c)
            print(f"Applying filter on {col_icp}")
            includes = [v.strip() for v in filter_icp if not v.strip().startswith("-")]
            excludes = [v[1:].strip() for v in filter_icp if v.strip().startswith("-")]
#             print(includes, excludes)
            if len(includes) > 0:
                df = df[(df[col_icp].isin(includes))]
            if len(excludes) > 0:
                df = df[~(df[col_icp].isin(excludes))]
            print("- Remaining deals:", len(df))
            print()
            
    # Create columns
    df["SCENARIO"] = pd.to_datetime(df["SQL_DATE"].str[:19]).dt.strftime("W%W-%Y")
    df["DEAL_NAME"] = df["ORG_NAME"] + " (" + df["PEOPLE_FULLNAME"] + ")"
    seniority_score = {
        "Entry-Level": 1,
        "Professional/Staff": 2,
        "Senior Professional/Staff": 3,
        "Lead/Supervisor": 4,
        "Manager": 5,
        "Senior Manager": 6,
        "Executive": 7,
        "Top Executive": 8
    }
    df["PEOPLE_SENIORITY_SCORE"] = df["PEOPLE_SENIORITY"].map(seniority_score).fillna(0)
    df["DEAL_SCORE"] = df["PEOPLE_INTERACTION_SCORE"].astype(int) + df["ORG_INTERACTION_SCORE"].astype(int) + df["PEOPLE_SENIORITY_SCORE"].astype(int)
    df["SQL_DATE"] = df["SQL_DATE"].str[:10]
    df["LAST_INTERACTION_DATE"] = df["LAST_INTERACTION_DATE"].str[:10]
    df["CRM_DEAL_ID"] = df["DEAL_NAME"].map(crm_ids).fillna("TBD").apply(lambda x: int(x) if type(x) == float else x)
    
    # Cleaning
    to_order = [
        "ENTITY",
        "SCENARIO",
        "DEAL_NAME",
        "SQL_DATE",
        "DEAL_SCORE",
        "LAST_INTERACTION_DATE",
        "PEOPLE_LAST_INTERACTION_DAYS",
        "PEOPLE_FULLNAME",
        "PEOPLE_OCCUPATION",
        "PEOPLE_SENIORITY",
        "PEOPLE_DEPARTMENT",
        "PEOPLE_NOTES",
        "MESSAGING_OPTIONS",
        "ORG_NAME",
        "ORG_STAFF_RANGE_NAME",
        "ORG_INDUSTRY",
        'ORG_COUNTRY',
        'PEOPLE_PROFILE_URL',
        'ORG_LINKEDIN_URL',
        "CRM_CONTACT_ID",
        "CRM_ORG_ID",
        "CRM_DEAL_ID"
    ]
    to_rename = {
        "SQL_DATE": "DEAL_CREATED_DATE",
        "LAST_INTERACTION_DATE": "PEOPLE_LAST_INTERACTION_DATE",
        "LAST_INTERACTION_DAYS": "PEOPLE_LAST_INTERACTION_DAYS"
    }
    df = df[to_order].rename(columns=to_rename).sort_values(by=["DEAL_CREATED_DATE", "DEAL_SCORE"], ascending=[False, False])
    return df.reset_index(drop=True)

db_deals = create_db_deals(
    df_init,
    df_contacts,
    df_entity,
    entity_name
)
print("🙌 Deals:", len(db_deals))
db_deals.head(3)

## Output

### Save data

In [ ]:
pdump(output_dir, db_deals, file_deals)

### Send data to Google Sheets spreadsheet

In [ ]:
df_check = pd.concat([db_deals.astype(str), df_init.astype(str)]).drop_duplicates(keep=False)
if len(df_check) > 0:
    gsheet.connect(spreadsheet_url).send(sheet_name=sheet_name_output, data=db_deals, append=False)
else:
    print("Noting to update in Google Sheets!")   